In [1]:
import requests
from dotenv import load_dotenv
import os


### Utilize historical flight data to predict the likelihood of delays for upcoming flights. Features can include departure and arrival airports, scheduled times, airline, and aircraft type. This project involves data preprocessing, model training, and deployment, offering a comprehensive MLOps experience.

- Try to pull to postgres from 2024-02-01 to 2024-04-31

In [2]:
load_dotenv()
# Access the variable
api_key = os.getenv("AVIATIONSTACK_API_KEY")

In [ ]:
# base_flights_url = "https://api.aviationstack.com/v1/flights"
# base_routes_url = "https://api.aviationstack.com/v1/routes"
# base_airports_url = "https://api.aviationstack.com/v1/airports"
# base_airlines_url = "https://api.aviationstack.com/v1/airlines"

params = {
    "access_key": api_key,
    "flight_status": None,  # optional
    "limit": None                    # optional
}

In [ ]:
flight_status = "active" ## schedueled, active, landed, cancelled, incident, diverted
flight_date = "2024-06-01" ## YYYY-MM-DD



'42bbbc0e4714cac81c219f982a3d332d'

In [4]:
BASE_URL = "https://api.aviationstack.com/v1/"

def call_aviationstack(endpoint: str, additional_params: dict = None):
    """
    Call the AviationStack API with the given endpoint and optional parameters.
    
    Args:
        endpoint (str): One of ["flights", "routes", "airports", "airlines"]
        additional_params (dict): Optional additional query parameters
        
    Returns:
        dict: JSON response from the API
    """
    url = f"{BASE_URL}{endpoint}"
    
    # Required access_key
    params = {
        "access_key": api_key
    }
    
    # Add optional params if provided
    if additional_params:
        params.update(additional_params)
    
    response = requests.get(url, params=params)
    
    return response.json()


In [ ]:
flights_data = call_aviationstack("flights",params)
airport_data = call_aviationstack("airports",params)
routes_data = call_aviationstack("routes",params)
airlines_data = call_aviationstack("airlines",params)
airplanes_data = call_aviationstack("airplanes",params)

In [10]:
base_url = "https://api.aviationstack.com/v1/flights"

params = {
    "access_key": api_key,
    "limit": 5,  # Optional: limit number of results
}

In [11]:
response = requests.get(base_url, params=params)

print(response.json())

{'pagination': {'limit': 5, 'offset': 0, 'count': 5, 'total': 415212}, 'data': [{'flight_date': '2025-04-30', 'flight_status': 'scheduled', 'departure': {'airport': 'Auckland International', 'timezone': 'Pacific/Auckland', 'iata': 'AKL', 'icao': 'NZAA', 'terminal': 'I', 'gate': None, 'delay': None, 'scheduled': '2025-04-30T00:25:00+00:00', 'estimated': '2025-04-30T00:25:00+00:00', 'actual': None, 'estimated_runway': None, 'actual_runway': None}, 'arrival': {'airport': 'Kuala Lumpur International Airport (klia)', 'timezone': 'Asia/Kuala_Lumpur', 'iata': 'KUL', 'icao': 'WMKK', 'terminal': '1', 'gate': None, 'baggage': None, 'scheduled': '2025-04-30T07:55:00+00:00', 'delay': None, 'estimated': None, 'actual': None, 'estimated_runway': None, 'actual_runway': None}, 'airline': {'name': 'Firefly', 'iata': 'FY', 'icao': 'FFM'}, 'flight': {'number': '7170', 'iata': 'FY7170', 'icao': 'FFM7170', 'codeshared': {'airline_name': 'malaysia airlines', 'airline_iata': 'mh', 'airline_icao': 'mas', 'fli

In [15]:
response.json()

{'pagination': {'limit': 5, 'offset': 0, 'count': 5, 'total': 415212},
 'data': [{'flight_date': '2025-04-30',
   'flight_status': 'scheduled',
   'departure': {'airport': 'Auckland International',
    'timezone': 'Pacific/Auckland',
    'iata': 'AKL',
    'icao': 'NZAA',
    'terminal': 'I',
    'gate': None,
    'delay': None,
    'scheduled': '2025-04-30T00:25:00+00:00',
    'estimated': '2025-04-30T00:25:00+00:00',
    'actual': None,
    'estimated_runway': None,
    'actual_runway': None},
   'arrival': {'airport': 'Kuala Lumpur International Airport (klia)',
    'timezone': 'Asia/Kuala_Lumpur',
    'iata': 'KUL',
    'icao': 'WMKK',
    'terminal': '1',
    'gate': None,
    'baggage': None,
    'scheduled': '2025-04-30T07:55:00+00:00',
    'delay': None,
    'estimated': None,
    'actual': None,
    'estimated_runway': None,
    'actual_runway': None},
   'airline': {'name': 'Firefly', 'iata': 'FY', 'icao': 'FFM'},
   'flight': {'number': '7170',
    'iata': 'FY7170',
    'ica

In [ ]:

# Example of how to use the API


querystring = {"flight_date":"2024-06-01", "limit":"1"}

In [7]:
flights_url

'https://api.aviationstack.com/v1/flights?access_key=42bbbc0e4714cac81c219f982a3d332d'

In [4]:
flights_url = "https://api.aviationstack.com/v1/flights?access_key=42bbbc0e4714cac81c219f982a3d332d"

In [ ]:
https://api.aviationstack.com/v1/flights?access_key=42bbbc0e4714cac81c219f982a3d332d

In [5]:
# Historical Flights

response = requests.get(flights_url)

print(response.json())

{'pagination': {'limit': 100, 'offset': 0, 'count': 100, 'total': 409482}, 'data': [{'flight_date': '2025-04-30', 'flight_status': 'scheduled', 'departure': {'airport': 'Auckland International', 'timezone': 'Pacific/Auckland', 'iata': 'AKL', 'icao': 'NZAA', 'terminal': 'I', 'gate': None, 'delay': None, 'scheduled': '2025-04-30T00:25:00+00:00', 'estimated': '2025-04-30T00:25:00+00:00', 'actual': None, 'estimated_runway': None, 'actual_runway': None}, 'arrival': {'airport': 'Kuala Lumpur International Airport (klia)', 'timezone': 'Asia/Kuala_Lumpur', 'iata': 'KUL', 'icao': 'WMKK', 'terminal': '1', 'gate': None, 'baggage': None, 'scheduled': '2025-04-30T07:55:00+00:00', 'delay': None, 'estimated': None, 'actual': None, 'estimated_runway': None, 'actual_runway': None}, 'airline': {'name': 'Firefly', 'iata': 'FY', 'icao': 'FFM'}, 'flight': {'number': '7170', 'iata': 'FY7170', 'icao': 'FFM7170', 'codeshared': {'airline_name': 'malaysia airlines', 'airline_iata': 'mh', 'airline_icao': 'mas', 

In [ ]:
      {
         "airport_name": "Anaa",
         "iata_code": "AAA",
         "icao_code": "NTGA",
         "latitude": "-17.05",
         "longitude": "-145.41667",
         "geoname_id": "6947726",
         "timezone": "Pacific/Tahiti",
         "gmt": "-10",
         "phone_number": null,
         "country_name": "French Polynesia",
         "country_iso2": "PF",
         "city_iata_code": "AAA"
      },